## 통계적 가설검정

In [22]:
import numpy as np
import pandas as pd
from scipy import stats

%precision 3
np.random.seed(1111)

In [23]:
from sklearn.datasets import load_boston

boston_houseprice_data = load_boston()

df = pd.DataFrame(
    data = boston_houseprice_data.data, 
    columns = boston_houseprice_data.feature_names
)
rooms = np.array(df['RM'])
sample = np.array(df['RM'])[:20]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

In [24]:
# 모평균, 표본평균
p_mean, s_mean = np.mean(rooms), np.mean(sample)
p_mean, s_mean

(6.285, 6.192)

In [25]:
# 모분산, 표본분산
p_var, s_var = np.var(rooms), np.var(sample, ddof=1) # 불편분산
p_var, s_var

(0.493, 0.232)

## 통계적 가설검정이란

통계적 가설검정 용어

- 2가지 가설
- 귀무가설 : 대립가설의 반대되는 개념 (기존의 정책)
- 대립가설 : 주장하고 싶은 가설 (세로운 정책)
- 귀무가설을 기각하다 = 귀무가설은 옳지 않다 (세로운 정책이 유의미하다)
- 귀무가설을 채택하다 = 귀무가설이 옳지 않다고 할 수 없다 (귀무가설이 옳다는 의미는 아님!)


- 유의하다 (Significant)
- 귀무가설을 바탕으로 표본을 통해 계산된 통계량이 특정 신뢰수준 범위 내에서 보기 드문 값인지 여부를 통해서 결정 (보기 드문 값이면 귀무가설 기각, 그렇지 않다면 귀무가설 채택)


- 기각역  : 귀무가설이 기각되는 구간
- 채택역 : 귀무가설이 채택되는 구간


- 유의수준(신뢰수준, Level of Significance) : 표본평균이 기각역에 들어갈 확률 알파
- 임계값(Critical Value) : 경계선상의 값
- 검정통계량(Test Statistic) : 검정에 사용될 통계량
- P값(p-value) : 검정통계량 기준으로 바깥 영역의 면적

### 통계적 가설검정의 흐름

In [26]:
# 임계값
rv = stats.norm(6, np.sqrt(0.2/20))
rv.isf(0.05)

6.164

In [27]:
z = (s_mean - 6) / np.sqrt(0.2/20)
z

1.919

In [28]:
rv = stats.norm()
rv.isf(0.95)

-1.645

In [29]:
# p 값
rv.cdf(z)

0.972

### 단측검정과 양측검정
- 단측검정 : 한 방향으로만 검정을 수행하는 방법
- 양측검정 : 양 방향으로 검정을 수행하는 방법
- 주의 : 기각역이 다르다
- 단측검정은 한쪽방향으로 기각역 범위는 a
- 양측검정은 양쪽방향으로 기각역 범위는 a/2

가설검정 두 가지 오류
- 제 1종 오류 : 귀무가설이 옳을 때 귀무가설을 기각하는 오류 (오탐, False Positive)
- 제 2종 오류 : 대립가설이 옳을 때 귀무가설을 채택하는 오류 (미탐, False Negative)
- Positive : 귀무가설 기각에 해당
- Negative : 귀무가설 기각의 실패에 해당

In [30]:
z = (s_mean - 6) / np.sqrt(0.2/20)
z

1.919

In [31]:
rv = stats.norm()
rv.interval(0.95)

(-1.960, 1.960)

In [32]:
rv.cdf(z) * 2

1.945

In [33]:
# 제 1종 오류 : 평균이 6인데, 6보다 크다라는 결론을 내리는 상황
rv = stats.norm(6, np.sqrt(0.2))

In [34]:
c = stats.norm().isf(0.05)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(20), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 6) / np.sqrt(0.2/20)
    if z > c:
        cnt += 1
cnt / n_samples

0.051

In [35]:
c

1.645

In [36]:
# 제 2종 오류 : 평균이 6이상인데, 6이상이라는 결론을 얻을 수 없는 상황
rv = stats.norm(6.15, np.sqrt(0.2)) # 비대칭정보 모평균 = 6.15

In [37]:
c = stats.norm().isf(0.05)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(20), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 6) / np.sqrt(0.2/20)
    if z <= c:
        cnt += 1
        
cnt / n_samples

0.567

In [38]:
c

1.645

### 가설검정

#### 정규분포의 모평균에 대한 검정 (모분산을 알고 있음)

In [39]:
def pmean_test(sample, mean0, p_var, alpha=0.05):
    s_mean = np.mean(sample)
    n = len(sample)
    rv = stats.norm()
    interval = rv.interval(1-alpha)

    z = (s_mean - mean0) / np.sqrt(p_var/n)
    if interval[0] <= z <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')

    if z < 0:
        p = rv.cdf(z) * 2
    else:
        p = (1 - rv.cdf(z)) * 2
    print(f'p값은 {p:.3f}')

In [40]:
# 유의수준 5%로 양측검정시, p값이 0.055로 0.05보다 크므로 귀무가설을 채택하게 됨
pmean_test(sample, 6, 0.2)

귀무가설을 채택
p값은 0.055


#### 정규분포의 모분산에 대한 검정

In [49]:
# 정규분포 모분산에 대한 검정
def pvar_test(sample, var0, alpha=0.05):
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.chi2(df=n-1)
    interval = rv.interval(1-alpha)
    
    y = (n-1) * u_var / var0
    if interval[0] <= y <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')

    if y < rv.isf(0.5):
        p = rv.cdf(y) * 2
    else:
        p = (1 - rv.cdf(y)) * 2
    print(f'p값은 {p:.3f}')

In [50]:
# 유의수준 5%로 양측검정시, p값이 0.047로서 0.05보다 작아 귀무가설을 기각하게 됨
pvar_test(sample, 0.5)

귀무가설을 기각
p값은 0.047


#### 정규분포의 모평균에 대한 검정(모분산을 알지 못함)
- 모분산을 알지 못하는 상황에서 정규분포의 모평균에 대한 검정은 1표본 t검정(1-sample t-test)이라고 한다

In [51]:
# 모분산 모를 때 정규분포 모평균에 대한 검정
def pmean_test(sample, mean0, alpha=0.05):
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1-alpha)

    t = (s_mean - mean0) / np.sqrt(u_var/n)
    if interval[0] <= t <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')

    if t < 0:
        p = rv.cdf(t) * 2
    else:
        p = (1 - rv.cdf(t)) * 2
    print(f'p값은 {p:.3f}')

In [52]:
# 유의수준 5%로 양측검정시, p값이 0.091로서 0.05보다 크므로 귀무가설을 채택하게 됨
pmean_test(sample, 6)

귀무가설을 채택
p값은 0.091


In [53]:
# 1표본 t검정 (1-sample t-test)
# scipy.stats의 ttest_1samp 모듈 활용하면 t 검정통계량과 p값을 얻을 수 있다
t, p = stats.ttest_1samp(sample, 6)
t, p


(1.781, 0.091)

### 2표본 문제에 관한 가설검정

- 2개 모집단에 관한 다양한 관계성 검정을 위한 문제로서, 2개 표본 각각의 대표값 사이에 차이가 있는지 검정을 통해 문제를 해결한다<br>
- 2표본 문제 4가지 검정 방법
- 1. 모집단 정규분포를 가정할 수 있는지
- 2. 데이터가 서로 대응이 있는지 or 독립인지<br>
- 대응비교 t 검정, 독립비교 t 검정, 윌콕슨의 부호순위검정, 만위트니의 U 검정<br>






#### 대응비교 t검정
- 대응하는 데이터가 존재하고, 데이터 차이에 정규분포를 가정할 수 있는 경우에 평균값 차이에 대한 검정 방법

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [54]:
# 근력운동 전/후 집중력 테스트 결과값 데이터 파악
training_rel = pd.read_csv('/content/drive/MyDrive/개인최윤아/Today_I_learn/5월/[교제]part9.-확률-및-통계-파이썬-코드파일-ipynb.-22.02.24-/[패스트캠퍼스-확률및통계] 11_training_rel.csv')
print(training_rel.shape)
training_rel.head()

(20, 2)


,전,후
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84


In [55]:
# 근력운동 전/후 집중력 테스트 결과값 차이 비교 (양측 검정)
training_rel['차'] = training_rel['후'] - training_rel['전']
training_rel.head()

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25


In [56]:
# 차이값들의 평균이 0인지 아닌지 여부를 검정하는 것과 동일
# scipy.stats의 ttest_1samp로 1표본 t검정 계산하기
t, p = stats.ttest_1samp(training_rel['차'], 0)
p

0.040

In [57]:
# 차이값 분포를 scipy.stats의 ttest_rel로 전/후 데이터 기반 동일한 검정 계산
t, p = stats.ttest_rel(training_rel['후'], training_rel['전'])
p
# p값이 0.04로서 0.05보다 작으므로 귀무가설은 기각되고, 근력운동이 집중력에 유의한 차이를 가져오는 것을 확인할 수 있다.

0.040

### 독립비교 t검정
- 대응하는 데이터가 없고, 독립된 2표본 모집단에 정규분포를 가정할 수 있는 경우의 평균값 차이에 대한 검정 방법


- 예시) A학급은 근력운동을 안하는 집단, B학급은 근력운동을 꾸준히 하는 집단, 두집단간 집중력 차이가 있는지 여부 파악하기 위해 독립적인 두 학급에서 집중력 테스트 결과를 생성

In [58]:
# A학급/B학급 집중력 테스트 결과값 데이터 파악
training_ind = pd.read_csv('/content/drive/MyDrive/개인최윤아/Today_I_learn/5월/[교제]part9.-확률-및-통계-파이썬-코드파일-ipynb.-22.02.24-/[패스트캠퍼스-확률및통계] 11_training_ind.csv')
print(training_ind.shape)
training_ind.head()

# A학급/B학급 집중력 테스트 결과값 차이 비교 (양측 검정)
# 대응이 없는 독립표본 데이터이므로 개별 데이터 간의 차이를 구하는 것은 의미가 없다

(20, 2)


,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


In [59]:
# 검정통계량은 t가 활용 자유도는 t분포를 따른다. -> 웰치의 방법
# stats.ttest_ind 함수 활용하면 간단히 계산 가능, equal_var=False를 지정하면 웰치의 방법으로 계산 가능
t, p = stats.ttest_ind(training_ind['A'], training_ind['B'],
                       equal_var=False)
p
# p값이 유의수준이 0.05보다 크기 때문에, 귀무가설이 채택된다

0.087

### 윌콕슨의 부호순위검정
- 대응하는 데이터에서 독립된 2표본 모집단에 정규분포를 가정할 수 없는 경우
- 윌콕슨의 부호순위검정 방법
- 1. 데이터간의 차이를 절대값을 기준으로 순위를 나타낸다
- 2. 차이의 부호가 마이너스인 것의 순위합 및 플러스인 것의 순위합을 고려하여 더 작은 쪽이 검정통계량이 되며, 임계값 기준보다 작으면 차이가 있다는 주장을 할 수 있다.
- 3. scipy.stats의 wilcoxon 함수를 활용하면, 부호의 순위합을 계산하고 표준화를 수행한 뒤 정규분포로 검정을 통해서 검정통계량 값이 약간 바뀌긴 하지만, 기본 원리에는 차이가 없다.

In [61]:
# 근력운동 전/후 집중력 테스트 결과값 데이터 파악
training_rel = pd.read_csv('/content/drive/MyDrive/개인최윤아/Today_I_learn/5월/[교제]part9.-확률-및-통계-파이썬-코드파일-ipynb.-22.02.24-/[패스트캠퍼스-확률및통계] 11_training_rel.csv')
toy_df = training_rel[:6].copy()
toy_df

,전,후
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84
5,45,37


In [62]:
diff = toy_df['후'] - toy_df['전']
toy_df['차'] = diff
toy_df

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25
5,45,37,-8


In [63]:
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,41,-18,5
1,52,63,11,3
2,55,68,13,4
3,61,59,-2,1
4,59,84,25,6
5,45,37,-8,2


In [64]:
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

(8, 13)

In [65]:
toy_df['후'] = toy_df['전'] + np.arange(1, 7)
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['차'] = diff
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,60,1,1
1,52,54,2,2
2,55,58,3,3
3,61,65,4,4
4,59,64,5,5
5,45,51,6,6


In [66]:
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

(0, 21)

In [67]:
toy_df['후'] = toy_df['전'] + [1, -2, -3, 4, 5, -6]
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['차'] = diff
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,60,1,1
1,52,50,-2,2
2,55,52,-3,3
3,61,65,4,4
4,59,64,5,5
5,45,39,-6,6


In [68]:
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

(11, 10)

In [69]:
T, p = stats.wilcoxon(training_rel['전'], training_rel['후'])
p

0.038

In [70]:
T, p = stats.wilcoxon(training_rel['후'] - training_rel['전'])
p

0.038

In [71]:
n = 10000
diffs = np.round(stats.norm(3, 4).rvs(size=(n, 20)))

In [72]:
cnt = 0
alpha = 0.05
for diff in diffs:
    t, p = stats.ttest_1samp(diff, 0)
    if p < alpha:
        cnt += 1
cnt / n

0.878

In [73]:
cnt = 0
alpha = 0.05
for diff in diffs:
    T, p = stats.wilcoxon(diff)
    if p < alpha:
        cnt += 1
cnt / n

0.869

### 만・위트니의 U검정
- 대응되는 데이터가 없는 독립되 2표본 모집단에 정규분포를 가정할 수 없는 경우 중앙값 차이에 대한 검정 방법으로서, 윌콕스의 순위합검정(Not 부호순위검정)이라고도 불린다.
- 2표본 데이터 전체에 대해서 값이 작은 순서대로 순위를 부여하고, 검정통계량은 한 표본 데이터의 순위합을 사용한다.
- scipy.stats의 mannwhitneyu 함수를 사용하여 U 검정을 할 수 있으며, 데이터는 2표본, 파라미터 alternative는 'two-sided'로 설정하면 된다.

In [74]:
# A학급/B학급 집중력 테스트 결과값 데이터 파악
training_ind = pd.read_csv('/content/drive/MyDrive/개인최윤아/Today_I_learn/5월/[교제]part9.-확률-및-통계-파이썬-코드파일-ipynb.-22.02.24-/[패스트캠퍼스-확률및통계] 11_training_ind.csv')
toy_df = training_ind[:5].copy()
toy_df

,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


In [75]:
rank = stats.rankdata(np.concatenate([toy_df['A'],
                                      toy_df['B']]))
rank_df = pd.DataFrame({'A': rank[:5],
                        'B': rank[5:10]}).astype(int)
rank_df

,A,B
0,3,5
1,6,8
2,1,9
3,10,4
4,2,7


In [76]:
n1 = len(rank_df['A'])
u = rank_df['A'].sum() - (n1*(n1+1))/2
u

7.000

In [77]:
rank_df = pd.DataFrame(np.arange(1, 11).reshape(2, 5).T,
                       columns=['A', 'B'])
rank_df

,A,B
0,1,6
1,2,7
2,3,8
3,4,9
4,5,10


In [78]:
u = rank_df['A'].sum() - (n1*(n1+1))/2
u

0.000

In [79]:
rank_df = pd.DataFrame(np.arange(1, 11).reshape(2, 5)[::-1].T,
                       columns=['A', 'B'])
rank_df

,A,B
0,6,1
1,7,2
2,8,3
3,9,4
4,10,5


In [80]:
u = rank_df['A'].sum() - (n1*(n1+1))/2
u

25.000

In [81]:
u, p = stats.mannwhitneyu(training_ind['A'], training_ind['B'],
                          alternative='two-sided')
p

0.059

### 카이제곱검정
- 두 변수 X, Y에 관해 'X와 Y가 독립이다' 라는 귀무가설을 기반으로 수행되는 검정
- 독립성 검정에는 카이제곱분포가 사용되기 때문에 카이제곱검정(Chi-square Test)라고도 불린다


In [82]:
ad_df = pd.read_csv(
    '/content/drive/MyDrive/개인최윤아/Today_I_learn/5월/[교제]part9.-확률-및-통계-파이썬-코드파일-ipynb.-22.02.24-/[패스트캠퍼스-확률및통계] 11_ad.csv'
)
n = len(ad_df)
print(n)
ad_df.head()
# 독립성 검정 (광고, 상품 구입 유무 데이터 활용)

1000


,광고,구입
0,B,하지 않았다
1,B,하지 않았다
2,A,했다
3,A,했다
4,B,하지 않았다


In [83]:
# 교차집계표(Cross Table) 작성 - 도수분포표의 2변수 버전
ad_cross = pd.crosstab(ad_df['광고'], ad_df['구입'])
ad_cross

구입,하지 않았다,했다
광고,,
A,351,49
B,549,51


In [84]:
# A와 B 각각의 광고를 통해 상품을 구입한 비율이 어떻게 되는지 알아보기
ad_cross['했다'] / (ad_cross['했다'] + ad_cross['하지 않았다'])

광고
A    0.1225
B    0.0850
dtype: float64

In [85]:
# 상품을 구입한 사람의 합계, 상품을 구입하지 않은 사람의 합계
n_not, n_yes = ad_cross.sum()
n_not, n_yes

(900, 100)

In [86]:
# 광고 A를 본 사람의 합계, 광고 B를 본 사람의 합계
n_adA, n_adB = ad_cross.sum(axis=1)
n_adA, n_adB

(400, 600)

In [87]:
# 독립성 검정 방법
# A 광고를 본 후 구입하게 될 기대도수는 40명, 실제로 관측된 데이터는 관측도수라고 한다.
ad_ef = pd.DataFrame({'했다': [n_adA * n_yes / n,
                              n_adB * n_yes / n],
                      '하지 않았다': [n_adA * n_not / n,
                                   n_adB * n_not / n]},
                      index=['A', 'B'])
ad_ef

,했다,하지 않았다
A,40.0,360.0
B,60.0,540.0


In [89]:
# 관측도수와 기대도수의 i번째 행, j번째 열의 성분들을 각각 관측도수, 기대도수라고 하면 
# 관측도수 기대도수 차이를 측정하여 아래의 검정통계량을 계산한다.
y = ((ad_cross - ad_ef) ** 2 / ad_ef).sum().sum()
y

3.750

In [90]:
# Y는 자유도가 1인 카이제곱분포를 근사적으로 따른다고 알려져 있어서 p값을 간단하게 구할 수 있다
rv = stats.chi2(1)
1 - rv.cdf(y)

0.053

In [91]:
# scipy.stats의 chi2_contingency함수를 사용하여 간단히 계산 가능
chi2, p, dof, ef = stats.chi2_contingency(ad_cross,
                                          correction=False)
chi2, p, dof # 검정통계량, 자유도, 기대도수

(3.750, 0.053, 1)

In [92]:
ef

array([[360.,  40.],
       [540.,  60.]])